# flickr8k

In [2]:
import pandas as pd
import numpy as np

In [211]:
# # official vilbertscore

# df = pd.read_csv("../results/flickr8k.csv")
# flickr_precision = df["precision"].to_list()
# flickr_recall = df["recall"].to_list()
# flickr_f1 = df["f1"].to_list()
    
# with open("../data/flickr8k/scores.pkl", "rb") as f:
#     human = pickle.load(f, encoding='latin1')
# human = [(x-1)/3 for x in human]

In [212]:
# custom vilbertscore
df = pd.read_csv("../data/processed/flickr8k/vilbertscore.csv")
flickr_precision = df["precision"].to_list()
flickr_recall = df["recall"].to_list()
flickr_f1 = df["f1"].to_list()
    
with open("../data/processed/flickr8k/annotations_avg.txt") as f:
    human = f.readlines()
    human = [ float(x.strip()) for x in human]
human = [(x-1)/3 for x in human]

In [213]:
df_final = pd.DataFrame([human, flickr_precision, flickr_recall, flickr_f1]).T
df_final = df_final.rename(columns={0:"human", 1:"precision", 2:"recall", 3:"f1"})
df_final.corr(method="kendall")

,human,precision,recall,f1
human,1.000000,0.565625,0.538817,0.566569
precision,0.565625,1.000000,0.745920,0.881924
recall,0.538817,0.745920,1.000000,0.863986
f1,0.566569,0.881924,0.863986,1.000000


# capeval1k

In [83]:
import pandas as pd
import numpy as np

In [84]:
df = pd.read_csv("../data/raw/capeval1k/capeval1k_all_metrics.csv", index_col=False)
df = df.drop(["Unnamed: 0"], axis=1)

In [85]:
df_vs = pd.read_csv("../data/processed/capeval1k/vilbertscore.csv", index_col=False)
df_vs_f1 = df_vs.drop(["precision", "recall"], axis=1)
df_vs_f1 = df_vs_f1.rename(columns={"f1":"vilbertscore"})

In [86]:
df_clip = pd.read_csv("../data/processed/capeval1k/clipcore.csv", index_col=False)

In [98]:
df_merge = pd.concat([df, df_vs_f1, df_clip], axis=1)
df_merge.corr(method="kendall")

,Human,b1,b4,rouge,meteor,cider,spice,umic,uniter,vilbertscore,clips,ref-clips
Human,1.000000,0.233354,0.237929,0.220282,0.288180,0.307134,0.279319,0.328392,0.298954,0.355049,0.270060,0.307734
b1,0.233354,1.000000,0.496043,0.754776,0.701735,0.628574,0.436869,0.225329,0.218976,0.442600,0.262556,0.310219
b4,0.237929,0.496043,1.000000,0.539040,0.550078,0.646618,0.394281,0.188711,0.174239,0.328372,0.210967,0.253250
rouge,0.220282,0.754776,0.539040,1.000000,0.670568,0.614655,0.435001,0.213505,0.210201,0.444972,0.241130,0.284343
meteor,0.288180,0.701735,0.550078,0.670568,1.000000,0.718166,0.525234,0.260220,0.261501,0.457552,0.312479,0.364420
cider,0.307134,0.628574,0.646618,0.614655,0.718166,1.000000,0.540338,0.239962,0.239123,0.455355,0.300344,0.358462
spice,0.279319,0.436869,0.394281,0.435001,0.525234,0.540338,1.000000,0.217499,0.220391,0.396213,0.285581,0.331238
umic,0.328392,0.225329,0.188711,0.213505,0.260220,0.239962,0.217499,1.000000,0.814626,0.277917,0.302507,0.316601
uniter,0.298954,0.218976,0.174239,0.210201,0.261501,0.239123,0.220391,0.814626,1.000000,0.261980,0.318232,0.324039
vilbertscore,0.355049,0.442600,0.328372,0.444972,0.457552,0.455355,0.396213,0.277917,0.261980,1.000000,0.332629,0.391940


In [97]:
df_merge_norm = (df_merge-df_merge.min())/(df_merge.max()-df_merge.min())
df_merge_norm.corr(method="pearson")

,Human,b1,b4,rouge,meteor,cider,spice,umic,uniter,vilbertscore,clips,ref-clips
Human,1.000000,0.329991,0.258226,0.315062,0.395745,0.407390,0.402388,0.466762,0.416887,0.531597,0.408319,0.466149
b1,0.329991,1.000000,0.518129,0.923728,0.882049,0.815689,0.640304,0.331593,0.264451,0.623452,0.372453,0.442223
b4,0.258226,0.518129,1.000000,0.586812,0.593348,0.766979,0.443048,0.204362,0.136561,0.333684,0.153590,0.203589
rouge,0.315062,0.923728,0.586812,1.000000,0.865323,0.814224,0.638343,0.319597,0.277789,0.620864,0.349783,0.414627
meteor,0.395745,0.882049,0.593348,0.865323,1.000000,0.885191,0.728753,0.382847,0.310550,0.634523,0.429134,0.502837
cider,0.407390,0.815689,0.766979,0.814224,0.885191,1.000000,0.731114,0.338542,0.285849,0.603324,0.386117,0.463534
spice,0.402388,0.640304,0.443048,0.638343,0.728753,0.731114,1.000000,0.328975,0.316416,0.581785,0.408458,0.480863
umic,0.466762,0.331593,0.204362,0.319597,0.382847,0.338542,0.328975,1.000000,0.732404,0.433821,0.450237,0.465763
uniter,0.416887,0.264451,0.136561,0.277789,0.310550,0.285849,0.316416,0.732404,1.000000,0.427024,0.396980,0.424291
vilbertscore,0.531597,0.623452,0.333684,0.620864,0.634523,0.603324,0.581785,0.433821,0.427024,1.000000,0.493288,0.577863


In [ ]:
# import json

# with open("../data/processed/capeval1k/cand.txt") as f:
#     cap_test = f.readlines()
#     cap_test = [x.strip() for x in cap_test]

# with open("../data/processed/capeval1k/capeval_clip_result.json") as f:
#     clip = json.load(f)
# clip[0]

# list_clips = []
# list_refclips = []
# for check, y in zip(cap_test, clip):
#     if check == y["candiate_caption"]:
#         list_clips.append(y["CLIP-S"])
#         list_refclips.append(y["RefCLIP-S"])
        
# df = pd.DataFrame([list_clips, list_refclips]).T
# df = df.rename(columns={0:"clips", 1:"ref-clips"})
# df.to_csv("../data/processed/capeval1k/clipcore.csv", index=False)